## 波斯顿房价预测

In [1]:
###required packages and load datasets
import numpy as np
import tensorflow as tf
from sklearn.datasets import load_boston

boston_data=load_boston()
dir(boston_data)

['DESCR', 'data', 'feature_names', 'target']

### z-scaling (加快梯度下降，是对normal-distribution数据的常用预处理手段)
## $x_z=\frac{X-\mu}{\sigma}$

In [25]:
data=boston_data.data
[n_samples,dim]=data.shape
data_mu=np.mean(data,axis=0)#axis=0表示纵向求均值
data_sigma=np.std(data,axis=0)
data_pre_z=(data-data_mu)/data_sigma
constant_feature=np.ones([n_samples,1])
data_z_scaled=np.concatenate((data_pre_z,constant_feature),axis=1)
tar=boston_data.target[:,np.newaxis]###tar=np.expend_dim(boston_data.target,axis=1)
num_samples,f_dimension=data_z_scaled.shape

In [8]:
a=np.array([[1,2,3,4],[3,2,3,5]])
print(np.mean(a),np.mean(a,axis=0),np.mean(a,axis=1))

2.875 [2.  2.  3.  4.5] [2.5  3.25]


In [13]:
b=np.zeros([2,1])
c=np.concatenate((a,b),axis=1)
c[0]

array([1., 2., 3., 4., 0.])

In [24]:
tar=boston_data.target[:,np.newaxis]
tar.shape

(506, 1)

In [31]:
with tf.name_scope("inputs"):
    x_=tf.placeholder(tf.float32,[None,f_dimension])
    y_=tf.placeholder(tf.float32,[None,1])

with tf.name_scope("Weights"):
    Weights=tf.Variable(tf.random_uniform([f_dimension,1],-1,1),name='W',dtype=tf.float32)

with tf.name_scope("loss"):
    loss=tf.reduce_mean(tf.square(tf.matmul(x_,Weights)-y_))
    tf.summary.scalar("loss",loss)
    
with tf.name_scope("Optimizer"): 
    train_step=tf.train.GradientDescentOptimizer(0.01).minimize(loss)
    
sess=tf.Session()
init=tf.global_variables_initializer()
merged=tf.summary.merge_all()
writer=tf.summary.FileWriter('./Bos_logs',sess.graph)
sess.run(init)

for i in range(1000):
    sess.run(train_step,feed_dict={x_:data_z_scaled,y_:tar})
    if((i+1)%50==0 and (i+1)>=50):
        cost=sess.run(loss,feed_dict={x_:data_z_scaled,y_:tar})
        print("Epochs:",(i+1),"loss:",cost)
        result=sess.run(merged,feed_dict={x_:data_z_scaled,y_:tar})
        writer=add_summary(result,i)
        
###在cmd中运行 tensorboard --logdir=./Desktop/Bos_logs

Epochs: 50 loss: 94.5403


InvalidArgumentError: You must feed a value for placeholder tensor 'inputs_2/Placeholder' with dtype float and shape [?,14]
	 [[Node: inputs_2/Placeholder = Placeholder[dtype=DT_FLOAT, shape=[?,14], _device="/job:localhost/replica:0/task:0/device:CPU:0"]()]]

Caused by op 'inputs_2/Placeholder', defined at:
  File "e:\python\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "e:\python\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "e:\python\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "e:\python\lib\site-packages\traitlets\config\application.py", line 658, in launch_instance
    app.start()
  File "e:\python\lib\site-packages\ipykernel\kernelapp.py", line 478, in start
    self.io_loop.start()
  File "e:\python\lib\site-packages\zmq\eventloop\ioloop.py", line 177, in start
    super(ZMQIOLoop, self).start()
  File "e:\python\lib\site-packages\tornado\ioloop.py", line 888, in 